# The Basis Mixer!

The previous notebooks explored the individual components of the Basis Function Models. In this notebook we will put everything together to render an expressive performance of a piece given its score.

In [ ]:
import numpy as np
from partitura import save_performance_midi, load_musicxml
from partitura.score import expand_grace_notes

from basismixer.performance_codec import get_performance_codec
from helper import load_model, compute_basis_from_xml
from helper.rendering import (post_process_predictions, 
                              path_to_trained_models)

## 0. Select a piece and trained models
We start by defining the piece we want to render (here is an example MusicXML file, but you can set your own!)

In [ ]:
xml_fn = './sample_data/musicxml/Mozart_K265_1.musicxml'

Now we set the path to the directory of the trained models from our previous notebook (or any other trained models). Following the directory structure from the previous notebook, each

In [ ]:
# models_dir = './put_your_models_here/'
models_dir = path_to_trained_models()

## 1. Load the Predictive Model

The function `load_model` looks for the configuration files and the saved model parameters in `models_dir`. This function will return an instance of `FullPredictiveModel`, which is a conveniece *meta-model* which contains a list of the individual models (in `model.models`).

In [ ]:
model, predicted_parameter_names = load_model(models_dir)

In [ ]:
# If you want to modify any of these default values just do
# model.default_values['parameter_name'] = value
# For example, for making the performance slower
beats_per_minute = 70
model.default_values['beat_period_std'] = 0.# 60.0 / beats_per_minute

## 2. Computing the Score Representation.

We load the input MusicXML file specified above and compute the input score representation using the Basis Functions specified by the model (in `model.input_names`).

In [ ]:
# Load MusicXML file
basis, part = compute_basis_from_xml(xml_fn, model.input_names)

## 3. Make Predict Performance

We make predictions of the performance using the models. , The `predict` method of `FullPredictiveModel` combines the predictions of the individual predictive models into a single output structured array, where the field names are the output parameters (specified in `model.output_names`).

In [ ]:
preds = model.predict(basis, part.note_array['onset'])
post_process_predictions(preds)
print('Field names:', preds.dtype.names)

## 4. Decode predictions

Finally, we decode the predictions using a `PerformanceCodec` and export the performance as a MIDI file. The output of the `decode` method is an instance of `partitura.performance.PerformedPart`.

In [ ]:
perf_codec = get_performance_codec(model.output_names)
predicted_ppart = perf_codec.decode(part, preds)

In [ ]:
# import matplotlib.pyplot as plt
# fig = plt.figure()
# plt.hist(predicted_ppart.note_array['p_duration'], 100); plt.show()

Finally, we can export the predictions of the model as a MIDI file!

In [ ]:
midi_fn = 'my_midi_output.mid'
save_performance_midi(predicted_ppart, midi_fn)

In [ ]:
from helper.rendering import render_midi

render_midi(midi_fn)

In [ ]:
%matplotlib notebook
from helper.plotting import plot_predictions
plot_predictions(preds, part.note_array['onset'], 
                 predicted_parameter_names)